In [1]:
# Load dependencies
import pandas as pd
import numpy as np
import gdal
from scipy.stats import gmean
import sys
sys.path.insert(0,'../../statistics_helper/')
from fraction_helper import *
from CI_helper import *
from excel_utils import *
pd.options.display.float_format = '{:,.1e}'.format

# Estimating the total biomass of deep subsurface fungi
To estimate the total biomass of fungi in the deep subsurface, we will focus on the terrestrial deep subsurface. As we detail in the SI Appendix, the abundance of fungi in the marine deep subsurface in likely not very high.

To estimate the total biomass of fungi in the terrestrial deep subsurface, we estimate the ratio between the biomass of fungi and prokaryotes in the deep terrestrial subsurface. We rely on two methodologies for estimating this ratio - direct counts and RNA hybridization.

## Direct counts
To estimate the ratio between the biomass of fungi and prokaryotes based on direct counts of cells we need to estimate two key parameters - the ratio between fungal and bacterial cells in the terrestrial deep subsurface, and the ratio between the carbon content of bacteria and fungi in the terrestrial deep subsurface.

### The ratio between the number of cells of fungi and prokaryotes cells in the terrestrial deep subsurface
To estimate the ratio between the total number of cells of fungi and prokaryotes in the terrestrial deep subsurface, we rely on two studies: [Sinclair & Ghiorse 1989](https://doi.org/10.1080/01490458909377847) and [Ekendahl et al.](http://dx.doi.org/10.1007/s00248-003-2008-5).

Sinclair & Ghiorse measured the density of colony forming units of fungi and prokaryotes per g dry weight of pristine aquifer samples. We use the geometric mean of the values from different sites as our best estimate for the ratio between the number of cells of fungi and prokaryotes in the terrestrial deep subsurface:

In [6]:
# Load the data from Sinclair & Ghiorse
sg_data = pd.read_excel('deep_subsurface_fungi.xlsx',skiprows=1)

# Calculate teh ratio between prokaryotes and fungi
sg_ratio = gmean(sg_data['Bacteria CFU [mL^-1]']/sg_data['Fungi [CFU mL^-1]'])

print('Our best estimate of the ratio between the number of cells of fungi and prokaryotes based on Sinclair & Ghiorse is ≈%.0e-fold more prokaryotes' %sg_ratio)

Our best estimate of the ratio between the number of cells of fungi and prokaryotes based on Sinclair & Ghiorse is ≈5e+04-fold more prokaryotes


The data in Sinclair & Ghiorse is based on culturing fungi and prokarytes, which might not represent the actual ratios *in situ*. We therefore use a second source, which measured the densities of fungal cells in the Fennoscandian Shield ([Ekendahl et al.](http://dx.doi.org/10.1007/s00248-003-2008-5)). Ekendahl et al. measured the density of fungal cells using both culturing-based methods and direct counts. They have found a density of between 0.01 and 1 fungal cells per mL of groundwater. We use the geometric mean of this range which is ≈0.1 fungal cells per mL as our best estimate of the concentration of fungal cells at the site. The densities of prokaryotes per mL of groundwater measured in the same site were measured by [Pedersen & Ekendahl 1990](http://dx.doi.org/10.1007/BF02543865). We use the geometric mean of the densities of cells measured by Pederesen & Ekendahl as representitive of the concentration of prokaryotes in the same site:

In [5]:
# Load the data from Pedersen & Ekendahl 
pe_data = pd.read_excel('deep_subsurface_fungi.xlsx','Pedersen & Ekendahl',skiprows=1)

# Calculate the mean concentration of prokaryotes in Pedersen & Ekendahl 
mean_prok_conc = gmean(pe_data['Density of prokaryotes [cells mL^-1]'])

# We use the geometric mean of the range reported in Ekendahl et al. of 0.01-1 cells mL^-1 
fungi_conc = 0.1
pe_ratio = mean_prok_conc/fungi_conc

print('Our best estimate of the ratio between the number of cells of fungi and prokaryotes based on Ekendahl et al. is ≈%.0e-fold more prokaryotes' %pe_ratio)

Our best estimate of the ratio between the number of cells of fungi and prokaryotes based on Ekendahl et al. is ≈2e+06-fold more prokaryotes


As our best estimate for the ratio between the number of cells of fungi and prokaryotes we use the geometric mean of the ratios we obtained based on Sinclair & Ghiorse and based on Ekendahl et al.:

In [7]:
best_fungi_ratio = gmean([sg_ratio,pe_ratio])

print('Our best estimate of the ratio between the number of cells of fungi and prokaryotes is ≈%.0e-fold more prokaryotes' %best_fungi_ratio)

Our best estimate of the ratio between the number of cells of fungi and prokaryotes is ≈3e+05-fold more prokaryotes


### The carbon content ratio between fungi and prokaryotes
We will now estimate the carbon content of single fungal cells in the terrestrial deep subsurface and compare it to our estimate of the characteristic carbon content of prokaryotes in the terrestrial deep subsurface. To estimate the carbon content of fungal cells, we use estimates of the volume of filamentous fungal cells and yeast cells. For filamentous fungi, we rely on [Veses et al.](http://dx.doi.org/10.1111/j.1365-2958.2008.06545.x), which measured the cell volume of *Candida albicans* at ≈100 $µm^3$. For yeast cells, we rely on measurements by Ekendahl et al. of yeast cells diameters from the Fennoscandian Shield. Ekendahl et al. reports cell diameters of 3 $µm^3$. We use the geometric mean of the volumes of filamentous cells and yeast cells as our best estimate of the volume of fungal cells:

In [9]:
# Volume of filamentous fungi from Veses et al.
filamentous_vol = 100

# Volume of yeast cells from the diameter reported by Ekendahl et al. and using a volume of a sphere
yeast_vol = 4*np.pi*(3/2)**3

# Caluclate the geometric mean of the volumes of filamentous fungi and yeast
best_vol = gmean([filamentous_vol,yeast_vol])

print('Our best estimate of the volume of fungal cells in the deep terrestrial subsurface is ≈%.0f µm^3' %best_vol)

Our best estimate of the volume of fungal cells in the deep terrestrial subsurface is ≈65 µm^3


To convert the biovolume of fungal cells to carbon content, we use a conversion factor of ≈120 fg C $µm^{-1}$ based on [Bakken & Olsen](http://aem.asm.org/cgi/pmidlookup?view=long&pmid=16346263).

In [27]:
# Convert the biovolume of fungi to carbon content
fungal_cc = best_vol*120e-15

print('Our best estimate for the volume of fungal cells in the terrestrial deep subsurface is ≈%.0f pg C' %(fungal_cc*1e12))

Our best estimate for the volume of fungal cells in the terrestrial deep subsurface is ≈8 pg C


 We divide our estimate for the carbon content of fungal cells by our estimate for the carbon content of prokaryotes in the terrestrial deep subsurface.

In [ ]:

# Use our estimate for the carbon content of prokaryotes in the terrestrial deep subsurface
prok_cc = pd.read_excel('../../bacteria_archaea/terrestrial_deep_subsurface/terrestrial_deep_subsurface_prok_biomass_estimate.xlsx').iloc[3,1]
cc_ratio = fungal_cc/prok_cc

print('Our best estimate for the carbon content ratio between fungi and prokaryotes is ≈%.0f-fold more carbon content in fungi' %cc_ratio)

To estimate the ratio between the biomass of fungi and prokaryotes in the terrestrial deep subsurface we multiply the ratio between the number of cells of fungi and prokaryotes and the ratio between the carbon contents of fungi and prokaryotes.

In [23]:
biomass_ratio = cc_ratio/best_fungi_ratio

print('Our estimate for the ratio between the biomass of fungi and prokaryotes in the terrestrial deep subsurface based on direct counts is ≈%.0e' %biomass_ratio)

Our estimate for the ratio between the biomass of fungi and prokaryotes in the terrestrial deep subsurface based on direct counts is ≈9e-04


##

In [47]:
# Load 
rRNA_data = pd.read_excel('deep_subsurface_fungi.xlsx','rRNA hybridization')
rRNA_study_mean = rRNA_data.groupby('Link').apply(lambda x: gmean(x['Fraction of eukaryotes out of total rRNA']))
best_rRNA_frac = gmean(rRNA_study_mean)

print('Our best estimate for the fraction of the total rRNA contributed by eukaryotes is ≈%.1f.' %(best_rRNA_frac*100))

CI_prod_prop(np.array([22,geo_CI_calc(np.array([biomass_ratio,best_rRNA_frac/2]))]))
CI_sum_prop(estimates=np.array([12,0.3,0.5]),mul_CIs=np.array([3.5,10,166]))

Our best estimate for the fraction of the total rRNA contributed by eukaryotes is ≈11.7.


5.044562502752089